In [445]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Alignment
from openpyxl import load_workbook
import time
import datetime
pd.options.mode.chained_assignment = None  # default='warn'

In [446]:
def combine(x):
    # Функция для группировки всех значений в строку разделенную ;
    return  ';'.join(x)

def check_unique(x):
    # Функция для нахождения разночтений в значениях
    temp_lst = x.split(';')
    temp_set = set(temp_lst)
    if 'nan' in temp_set:
        return 'Не заполнены значения площади лесотаксационного выдела!!!'
    else:
        return 'Все в порядке' if len(temp_set) == 1 else 'Ошибка!!!'

def main_check_unique(x):
    # Функция для проверки корректности заполнения площади выдела
    temp_str = ';'.join(x) # Склеиваем все значения
    temp_lst = temp_str.split(';') # Создаем список разбивая по ;
    temp_set = set(temp_lst) # Превращаем в множество
    if len(temp_set) > 1: # Если длина множества больше 1 то есть погрешности
        return 0
    elif'nan' in temp_set:# если есть нан то не заполнены площади выдела
        return 0
    else:# Если все в порядке то возвращаем единственный элемент списка
        return convert_to_float(temp_lst[0])

def convert_to_float(x):
    """
    Функция для конвертирования строки в флоат.
    :param x: строка
    :return:
    """
    # очищаем от пробелов и возможных заполнения ячейки пробелом
    
    try:
        return float(x)
    except ValueError:
        return 0
def convert_to_int_transfer(cell):
    """
    Функция для конвертации в int
    """
    try:
        return int(cell)
    except ValueError:
        return 0
    
def create_doc_convert_date(cell):
    """
    Функция для конвертации даты при создании документов
    :param cell:
    :return:
    """
    try:
        string_date = datetime.datetime.strftime(cell, '%d.%m.%Y')
        return string_date
    except ValueError:
        return ''

# def clean_column(df,column):
#     """
#     Функция для очистки от колонки от пустых значений
#     """
#     df[column] = df[column].astype(str) # Приводим к строковому виду
#     df[column] = df[column].apply(lambda x:x.replace('nan','Отсутствует'))
#     df[column] = df[column].apply(lambda x:x.strip()) # очищаем от пробельных символов(на случай если в ячейке стоит просто пробел или несколько пробелов)
#     df[column] = df[column].apply(lambda x:x if x else 'Отсутствует') # Если пустая строка то заменяем на значение Не заполнено

def clean_column(df,lst_column):
    """
    Функция для очистки от колонки от пустых значений
    """
    for column in lst_column:
        df[column].fillna('Отсутствует',inplace=True)
        df[column] = df[column].astype(str) # Приводим к строковому виду
        df[column] = df[column].apply(lambda x:x.replace('nan','Отсутствует'))
        df[column] = df[column].apply(lambda x:x.strip()) # очищаем от пробельных символов(на случай если в ячейке стоит просто пробел или несколько пробелов)
        df[column] = df[column].apply(lambda x:x if x else 'Отсутствует') # Если пустая строка то заменяем на значение Не заполнено


def prepare_column_to_int(cell):
    """
    Функция для обработки ячеек в инт, чтобы они в экселе потом не отображались как строки
    """
    try:
        float_value = float(cell)
        int_value = int(float_value)
        return int_value
    except ValueError:
        return cell

In [447]:
region = 1

# file_transfer_to_upp = 'data/Сравнение таблиц и присвоение данных (площади)/Таблица 3 с дополнительными колонками..xlsx'
# file_transfer_reestr = 'data/Сравнение таблиц и присвоение данных (площади)/2022-10-27_64_Реестр УПП с дополнительными колонками.xlsx'

file_transfer_to_upp = 'data/Сравнение таблиц и присвоение данных (площади)/Якутия октябрь_Таблица 3с доп. колонками.xlsx'
file_transfer_reestr = 'data/Сравнение таблиц и присвоение данных (площади)/Якутия 2022-11-27_72_Реестр УПП по новой форме.xlsx'

path_to_end_folder = 'data/'
header_table3 = 'data/Сравнение таблиц и присвоение данных (площади)/Заголовок Таблица 3 от 08.11.xlsx'

In [448]:
# Создаем список колонок которые нужно загрузить
use_cols_upp = list(range(11))
use_cols_table3 = list(range(34))

# Загружаем датафреймы
# При загрузке делаем строковыми колонки квартал,выдел,год лесоустройства, чтобы потом не получить неприятность при мердже
df = pd.read_excel(file_transfer_reestr,skiprows=8,usecols=use_cols_upp,dtype={4:str,5:str,11:str})# Реестр УПП 
df_table_3 = pd.read_excel(file_transfer_to_upp,skiprows=6,usecols=use_cols_table3,dtype={7:str,8:str,9:str,15:str}) # Таблица 3

In [449]:
df.head(3)

,1,2,3,4,5,6,7,8,9,10,11
0,Алданское,Алданское,NaN,21,20,74.0,NaN,1.3,NaN,NaN,1983
1,Алданское,Алданское,NaN,21,20,74.0,NaN,2.9,NaN,NaN,1983
2,Алданское,Алданское,NaN,21,21,31.0,NaN,4.2,NaN,NaN,1983


In [450]:
df_table_3.head(3)

,1,2,3,4,5,6,7,8,9,10,...,25,26,27,28,29,30,31,32,33,34
0,2022-09-29,NaN,1.0,72.0,Намское,Хомустахское,NaN,323,8,100.0,...,более 40%,более 40%,2022-05-16 00:00:00,1.0,0.0,1.0,103.0,NaN,1.0,2021.0
1,2022-09-29,NaN,0.3,72.0,Намское,Хомустахское,NaN,323,8,200.0,...,более 40%,более 40%,2022-05-16 00:00:00,1.0,0.0,1.0,103.0,NaN,0.3,2021.0
2,2022-09-29,NaN,5.3,72.0,Намское,Хомустахское,NaN,323,8,300.0,...,более 40%,более 40%,2022-05-16 00:00:00,1.0,0.0,1.0,103.0,NaN,5.3,2021.0


In [451]:
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df.columns = list(map(str,list(df.columns)))
# на всякий случай очищаем от пробельных символов
df.columns = list(map(lambda x:x.replace(" ",""),list(df.columns)))

"""
Соответствие названий колонок используемым в программе номерам колонок
Лесничество -1
Участковое лесничество- 2
Урочище - 3
Номер лесного квартала -4
Номер лесотаксационного выдела -5
Площадь лесотаксационного выдела, га - 6
Год лесоустройства -11 или 15 для таблицы 3
"""

df.rename(columns={'1':'Лесничество','2':'Участковое лесничество','3':'Урочище','4':'Номер лесного квартала','5':'Номер лесотаксационного выдела',
                  '6':'Площадь лесотаксационного выдела, га','11':'Год лесоустройства'},inplace=True)




clean_column(df,['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела','Год лесоустройства']) # очищаем от пустых строк и пробелов

# Обрабатываем колонки таблицы 3
# Приводим названия колонок к строковому виду, чтобы избежать возможных проблем с названиями колонок
df_table_3.columns = list(map(str,list(df_table_3.columns)))

# Очищаем на всякий случай от пробельных символов
df_table_3.columns = list(map(lambda x:x.strip(),list(df_table_3.columns)))

# Очищаем от нан и пустых строк
clean_column(df_table_3,['5','6','7','8','9','15'])

In [452]:
# Бурятия
if region == 0:
    df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].astype(str)

    df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].apply(
        lambda x: x.replace(',', '.'))

    # Группируем
    df_upp = df.groupby(['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                           'Номер лесотаксационного выдела']).agg({'Площадь лесотаксационного выдела, га': main_check_unique,
                                                                   })

    df_upp=df_upp.astype(str)


    # Извлекаем индексы в колонки
    df_upp = df_upp.reset_index()

    # делаем строковыми первые 5 колонок в датафрейме Реестр УПП
    df_upp[['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']] = df_upp[['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']].astype(str)

    # Добавляем разделители для колонок квартала и выдела
    df_upp['Номер лесного квартала'] = df_upp['Номер лесного квартала'].apply(lambda x:'кв.' + x) # Добавляем разделитель квартал
    df_upp['Номер лесотаксационного выдела'] = df_upp['Номер лесотаксационного выдела'].apply(lambda x:'в.' + x) # Добавляем разделитель выдел

    # Создаем колонку с идентификатором
    df_upp['ID'] = df_upp.loc[:,['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела'] ].sum(axis=1)
    df_upp['ID'] = df_upp['ID'].apply(lambda x:x.replace(' ',''))
    df_upp['ID'] = df_upp['ID'].apply(lambda x:x.replace('nan',''))



    # делаем строковыми колонки по которым будем соединять в датафрейме 3 таблицы
    df_table_3[['5','6','7','8','9']] = df_table_3[['5','6','7','8','9']].astype(str)
    # Добавляем разделители для колонок квартала и выдела
    df_table_3['8'] = df_table_3['8'].apply(lambda x:'кв.' + x) # Добавляем разделитель квартал
    df_table_3['9'] = df_table_3['9'].apply(lambda x:'в.' + x) # Добавляем разделитель выдел

    # Создаем колонку с идентификатором
    df_table_3['ID'] = df_table_3.loc[:,['5','6','7','8','9'] ].sum(axis=1)
    df_table_3['ID'] = df_table_3['ID'].apply(lambda x:x.replace(' ',''))
    df_table_3['ID'] = df_table_3['ID'].apply(lambda x:x.replace('nan',''))

    # Мержим по полю айди
    merged_df = pd.merge(df_upp,df_table_3,how='outer',left_on='ID',right_on='ID',indicator=True)



    # Заменяем площадь в таблице 3 в случае если  такой же выдел есть в реестре
    merged_df['Площадь'] = np.where(merged_df['_merge'] == 'both',merged_df['Площадь лесотаксационного выдела, га'],merged_df['10'])

    # Приводим к строковому виду чтобы очистить от разделителей кв. и в.
    merged_df['8'] = merged_df['8'].astype(str)
    merged_df['9'] = merged_df['9'].astype(str)
    # Очищаем от разделителей

    merged_df['8'] = merged_df['8'].apply(lambda x:x.replace('кв.',''))
    merged_df['9'] = merged_df['9'].apply(lambda x:x.replace('в.',''))
    
    #Очищаем от строковых nan
    merged_df[['5','6','7','8','9']] = merged_df[['5','6','7','8','9']].apply(lambda x:x.replace('nan',np.nan))

    # Приводим к инту колонки чтобы отсортировать их

    merged_df['8'] = merged_df['8'].apply(prepare_column_to_int)
    merged_df['9'] = merged_df['9'].apply(prepare_column_to_int)

    merged_df.sort_values(['5','6','7','8','9'],inplace=True)



    # Создаем документ
    wb = openpyxl.Workbook()
    # создаем листы
    ren_sheet = wb['Sheet']
    ren_sheet.title = 'Нет в таблице 3'
    wb.create_sheet(title='Нет в реестре УПП', index=1)
    wb.create_sheet(title='Выделы с обновленной площадью', index=2)

    # Записываем каждый датафрейм в соответсвующий лист
    # Записи реестра которых нет в таблице 3
    left_df = merged_df[merged_df['_merge'] == 'left_only']
    left_df.drop(left_df.iloc[:,7:],axis=1,inplace=True)



    for r in dataframe_to_rows(left_df, index=False, header=True):
        wb['Нет в таблице 3'].append(r)
    wb['Нет в таблице 3'].column_dimensions['A'].width = 30
    wb['Нет в таблице 3'].column_dimensions['B'].width = 30




    # Записываем каждый датафрейм в соответсвующий лист
    # Записи которые есть только в таблице 3
    right_df = merged_df[merged_df['_merge'] == 'right_only']
    right_df.drop(right_df.iloc[:,:6],axis=1,inplace=True)

    # Удаляем лишние колонки
    right_df.drop(right_df.iloc[:,10:],axis=1,inplace=True)

    right_df.drop(columns=['1','2','3','4'],axis=1,inplace=True)

    # Переименовываем колонки
    right_df.columns = ['ID','Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела']



    # Записываем на лист
    for r in dataframe_to_rows(right_df, index=False, header=True):
        wb['Нет в реестре УПП'].append(r)

    wb['Нет в реестре УПП'].column_dimensions['A'].width = 30
    wb['Нет в реестре УПП'].column_dimensions['B'].width = 30
    wb['Нет в реестре УПП'].column_dimensions['C'].width = 30



    # Записываем каждый датафрейм в соответсвующий лист
    both_df = merged_df[merged_df['_merge'] == 'both']

    both_df.drop(both_df.iloc[:,17:-1],axis=1,inplace=True)

    both_df.drop(columns=['1','2','3','4','5','6','7','8','9','Площадь лесотаксационного выдела, га'],axis=1,inplace=True)

    # # Очищаем от пробелов и строки названия урочища не заполнено
    # both_df['Урочище'] = both_df['Урочище'].apply(lambda x:x.replace('Название урочища не заполнено',''))
    # both_df['Урочище'] = both_df['Урочище'].apply(lambda x:x.strip())

    both_df.rename(columns={'Площадь':'Площадь из реестра УПП','10':'Старая площадь из таблицы 3'},inplace=True)

    for r in dataframe_to_rows(both_df, index=False, header=True):
        wb['Выделы с обновленной площадью'].append(r)

    wb['Выделы с обновленной площадью'].column_dimensions['A'].width = 30
    wb['Выделы с обновленной площадью'].column_dimensions['B'].width = 30
    wb['Выделы с обновленной площадью'].column_dimensions['G'].width = 20
    wb['Выделы с обновленной площадью'].column_dimensions['H'].width = 20


    # Получаем текущую дату
    current_time = time.strftime('%H_%M_%S %d.%m.%Y')
    wb.save(f'{path_to_end_folder}/Бурятия Данные для проверки корректности обновления площадей в таблице 3 от {current_time}.xlsx')

    # Создаем новую таблицу 3 с обновленными данными
    out_table3_df = merged_df[merged_df['_merge'] != 'left_only'] # отбираем общие значения и те что уже были в таблице 3

    # Присваиваем колонке 10 значение колонки созданной колонки 10
    out_table3_df['10'] = out_table3_df['Площадь']

    out_table3_df.drop(columns=['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела','Площадь лесотаксационного выдела, га','ID','_merge','Площадь'],inplace=True)

    # Приводим даты к нормальному виду ДД.ММ.ГГГГ
    out_table3_df['1'] = pd.to_datetime(out_table3_df['1'],errors='coerce',dayfirst=True)
    out_table3_df['22'] = pd.to_datetime(out_table3_df['22'],errors='coerce',dayfirst=True)
    out_table3_df['27'] = pd.to_datetime(out_table3_df['27'],errors='coerce',dayfirst=True)
    out_table3_df['1'] = out_table3_df['1'].apply(create_doc_convert_date)
    out_table3_df['22'] = out_table3_df['22'].apply(create_doc_convert_date)
    out_table3_df['27'] = out_table3_df['27'].apply(create_doc_convert_date)
    
    # Приводим год лесоустройства к инту
    out_table3_df['15'] = out_table3_df['15'].apply(prepare_column_to_int)

    # Получаем текущую дату
    current_time = time.strftime('%H_%M_%S %d.%m.%Y')
    # Сохраняем отчет
    # Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
    itog = load_workbook(header_table3) # Создаем объект
    # Записываем результаты
    for row in dataframe_to_rows(out_table3_df,index=False,header=False):
        itog['Лист1'].append(row)

    itog.save(f'{path_to_end_folder}/Бурятия Таблица 3 с присвоенными площадями из реестра УПП от {current_time}.xlsx')

# Якутия    
elif region == 1:
    

    df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].astype(str)

    df['Площадь лесотаксационного выдела, га'] = df['Площадь лесотаксационного выдела, га'].apply(
        lambda x: x.replace(',', '.'))

    # Группируем
    df_upp = df.groupby(['Лесничество', 'Участковое лесничество', 'Урочище', 'Номер лесного квартала',
                           'Номер лесотаксационного выдела','Год лесоустройства']).agg({'Площадь лесотаксационного выдела, га': main_check_unique,
                                                                   })

    # Извлекаем индексы в колонки
    df_upp = df_upp.reset_index()

    df_upp=df_upp.astype(str)


    # Добавляем разделители для колонок квартала и выдела, год лесоустройства
    df_upp['Номер лесного квартала'] = df_upp['Номер лесного квартала'].apply(lambda x:'кв.' + x) # Добавляем разделитель квартал
    df_upp['Номер лесотаксационного выдела'] = df_upp['Номер лесотаксационного выдела'].apply(lambda x:'в.' + x) # Добавляем разделитель выдел
    df_upp['Год лесоустройства'] = df_upp['Год лесоустройства'].apply(lambda x:'г.' + x) # Добавляем разделитель год

    # Создаем колонку с идентификатором
    df_upp['ID'] = df_upp.loc[:,['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела','Год лесоустройства'] ].sum(axis=1)
    df_upp['ID'] = df_upp['ID'].apply(lambda x:x.replace(' ',''))
    df_upp['ID'] = df_upp['ID'].apply(lambda x:x.replace('nan',''))

    # делаем строковыми колонки по которым будем соединять в датафрейме 3 таблицы
    df_table_3[['5','6','7','8','9','15']] = df_table_3[['5','6','7','8','9','15']].astype(str)
    # Добавляем разделители для колонок квартала и выдела
    df_table_3['8'] = df_table_3['8'].apply(lambda x:'кв.' + x) # Добавляем разделитель квартал
    df_table_3['9'] = df_table_3['9'].apply(lambda x:'в.' + x) # Добавляем разделитель выдел
    df_table_3['15'] = df_table_3['15'].apply(lambda x:'г.' + x) # Добавляем разделитель год

    # Создаем колонку с идентификатором
    df_table_3['ID'] = df_table_3.loc[:,['5','6','7','8','9','15'] ].sum(axis=1)
    df_table_3['ID'] = df_table_3['ID'].apply(lambda x:x.replace(' ',''))
    df_table_3['ID'] = df_table_3['ID'].apply(lambda x:x.replace('nan',''))

    # Мержим по полю айди
    merged_df = pd.merge(df_upp,df_table_3,how='outer',left_on='ID',right_on='ID',indicator=True)



    # Заменяем площадь в таблице 3 в случае если  такой же выдел есть в реестре
    merged_df['Площадь'] = np.where(merged_df['_merge'] == 'both',merged_df['Площадь лесотаксационного выдела, га'],merged_df['10'])



    # Приводим к строковому виду чтобы очистить от разделителей кв. и в.
    merged_df['8'] = merged_df['8'].astype(str)
    merged_df['9'] = merged_df['9'].astype(str)
    merged_df['15'] = merged_df['15'].astype(str)
    # Очищаем от разделителей

    merged_df['8'] = merged_df['8'].apply(lambda x:x.replace('кв.',''))
    merged_df['9'] = merged_df['9'].apply(lambda x:x.replace('в.',''))
    merged_df['15'] = merged_df['15'].apply(lambda x:x.replace('г.',''))

    #Очищаем от строковых nan
    merged_df[['5','6','7','8','9','15']] = merged_df[['5','6','7','8','9','15']].apply(lambda x:x.replace('nan',np.nan))


    # Приводим к инту колонки чтобы отсортировать их

    merged_df['8'] = merged_df['8'].apply(prepare_column_to_int)
    merged_df['9'] = merged_df['9'].apply(prepare_column_to_int)
    merged_df['15'] = merged_df['15'].apply(prepare_column_to_int)
    merged_df.sort_values(['5','6','7','8','9','15'],inplace=True)


    # Создаем документ
    wb = openpyxl.Workbook()
    # создаем листы
    ren_sheet = wb['Sheet']
    ren_sheet.title = 'Нет в таблице 3'
    wb.create_sheet(title='Нет в реестре УПП', index=1)
    wb.create_sheet(title='Выделы с обновленной площадью', index=2)



    # Записываем каждый датафрейм в соответсвующий лист
    # Записи реестра которых нет в таблице 3
    left_df = merged_df[merged_df['_merge'] == 'left_only']
    left_df.drop(left_df.iloc[:,8:],axis=1,inplace=True)

    for r in dataframe_to_rows(left_df, index=False, header=True):
        wb['Нет в таблице 3'].append(r)
    wb['Нет в таблице 3'].column_dimensions['A'].width = 30
    wb['Нет в таблице 3'].column_dimensions['B'].width = 30


    # Записываем каждый датафрейм в соответсвующий лист
    # Записи которые есть только в таблице 3
    right_df = merged_df[merged_df['_merge'] == 'right_only']

    right_df.drop(right_df.iloc[:,:7],axis=1,inplace=True)

    # Удаляем лишние колонки
    right_df.drop(right_df.iloc[:,16:],axis=1,inplace=True)

    right_df.drop(columns=['1','2','3','4','10','11','12','13','14',],axis=1,inplace=True)



    # Переименовываем колонки
    right_df.columns = ['ID','Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела','Год лесоустройства']

    # Записываем на лист
    for r in dataframe_to_rows(right_df, index=False, header=True):
        wb['Нет в реестре УПП'].append(r)

    wb['Нет в реестре УПП'].column_dimensions['A'].width = 30
    wb['Нет в реестре УПП'].column_dimensions['B'].width = 30
    wb['Нет в реестре УПП'].column_dimensions['C'].width = 30


    # Записываем каждый датафрейм в соответсвующий лист
    both_df = merged_df[merged_df['_merge'] == 'both']

    both_df.drop(both_df.iloc[:,23:-1],axis=1,inplace=True)

    both_df.drop(columns=['1','2','3','4','5','6','7','8','9','11','12','13','14','15','Площадь лесотаксационного выдела, га'],axis=1,inplace=True)

    both_df.rename(columns={'Площадь':'Площадь из реестра УПП','10':'Старая площадь из таблицы 3'},inplace=True)

    for r in dataframe_to_rows(both_df, index=False, header=True):
        wb['Выделы с обновленной площадью'].append(r)

    wb['Выделы с обновленной площадью'].column_dimensions['A'].width = 30
    wb['Выделы с обновленной площадью'].column_dimensions['B'].width = 30
    wb['Выделы с обновленной площадью'].column_dimensions['G'].width = 20
    wb['Выделы с обновленной площадью'].column_dimensions['H'].width = 20


    # Получаем текущую дату
    current_time = time.strftime('%H_%M_%S %d.%m.%Y')
    wb.save(f'{path_to_end_folder}/Саха (Якутия) Данные для проверки корректности обновления площадей в таблице 3 от {current_time}.xlsx')

    # Создаем новую таблицу 3 с обновленными данными
    out_table3_df = merged_df[merged_df['_merge'] != 'left_only'] # отбираем общие значения и те что уже были в таблице 3

    # Присваиваем колонке 10 значение колонки созданной колонки 10
    out_table3_df['10'] = out_table3_df['Площадь']

    out_table3_df.drop(columns=['Лесничество','Участковое лесничество','Урочище','Номер лесного квартала','Номер лесотаксационного выдела','Площадь лесотаксационного выдела, га','ID','_merge','Площадь','Год лесоустройства'],inplace=True)






    # Приводим даты к нормальному виду ДД.ММ.ГГГГ
    out_table3_df['1'] = pd.to_datetime(out_table3_df['1'],errors='coerce',dayfirst=True)
    out_table3_df['22'] = pd.to_datetime(out_table3_df['22'],errors='coerce',dayfirst=True)
    out_table3_df['27'] = pd.to_datetime(out_table3_df['27'],errors='coerce',dayfirst=True)
    out_table3_df['1'] = out_table3_df['1'].apply(create_doc_convert_date)
    out_table3_df['22'] = out_table3_df['22'].apply(create_doc_convert_date)
    out_table3_df['27'] = out_table3_df['27'].apply(create_doc_convert_date)

    # Приводим год лесоустройства к инту
    out_table3_df['15'] = out_table3_df['15'].apply(prepare_column_to_int)

    # Получаем текущую дату
    current_time = time.strftime('%H_%M_%S %d.%m.%Y')
    # Сохраняем отчет
    # Для того чтобы увеличить ширину колонок для удобства чтения используем openpyxl
    itog = load_workbook(header_table3) # Создаем объект
    # Записываем результаты
    for row in dataframe_to_rows(out_table3_df,index=False,header=False):
        itog['Лист1'].append(row)

    itog.save(f'{path_to_end_folder}/Саха (Якутия) Таблица 3 с присвоенными площадями из реестра УПП от {current_time}.xlsx')
    
    
    #